In [25]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from scipy.integrate import quad
from fractions import Fraction

matrix = []
matrisABNs = []
PERIODO_TOTAL = 0
OMEGA = 0
PI = 'π'

def reinicio():
    matrix = []
    matrisABNs = []
    PERIODO_TOTAL = 0
    OMEGA = 0
    PI = 'π'
    NRetorno = 0


def func_input():
    #matrix = []
    truncadoGen = 0
    while True:
        function = input("Ingresar valor o la definición de la función/Enter para cálculo de Sumatoria: ")
        if (function == "" and (not not matrix)):
            break
        else:
          if function == "SALIR":
            truncadoGen = -1
            break
          if function != "":
            a = input("Inicio de Intervalo: ")
            b = input("Fin de Intervalo: ")
            a = sp.sympify(a)
            b = sp.sympify(b)
            function = sp.sympify(function)  # Convierte la entrada en una expresión simbólica
            slice_array = [function, float(a), float(b)]
            matrix.append(slice_array)
    return truncadoGen

#------------------------ENERGIA------------------------
def energia(f, Ti_trozo,Tf_trozo):
    func_trozo = lambda x: (f.subs(sp.symbols('t'), x))**2  # Utiliza f como una función simbólica
    integral_result, _ = quad(func_trozo, Ti_trozo,Tf_trozo)
    return integral_result

def calcular_energia():
    energia_total = 0  # Inicializa la energía total en cero
    for i in range(len(matrix)):
        #periodo_trozo = abs(float(matrix[i][2]) - float(matrix[i][1]))
        energia_total += energia(matrix[i][0], float(matrix[i][1]),float(matrix[i][2]))  # Acumula la energía en cada iteración
    return energia_total

# VERIFICAR LIMITES DE INTEGRACION
#-------------Terminos de de la serie de Fourier-----------
def fourier_a0(f, Ti_trozo,Tf_trozo):
    PERIODO_TOTAL = np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2]))
    func_trozo = lambda x: f.subs(sp.symbols('t'), x)
    integral_result, _ = quad(func_trozo, Ti_trozo, Tf_trozo)
    a0 = (1 / PERIODO_TOTAL) * integral_result
    return a0

def fourier_an(f, Ti_trozo,Tf_trozo, n):
    PERIODO_TOTAL = np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2]))
    omega_n = 2 * np.pi * n / PERIODO_TOTAL
    func_trozo = lambda x: f.subs(sp.symbols('t'), x) * np.cos(omega_n * x)
    integral_result, _ = quad(func_trozo, Ti_trozo, Tf_trozo)
    an = (2 / PERIODO_TOTAL) * integral_result
    return an

def fourier_bn(f, Ti_trozo,Tf_trozo, n):
    PERIODO_TOTAL = np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2]))
    omega_n = 2 * np.pi * n / PERIODO_TOTAL
    func_trozo = lambda x: f.subs(sp.symbols('t'), x) * np.sin(omega_n * x)
    integral_result, _ = quad(func_trozo, Ti_trozo, Tf_trozo)
    bn = (2 / PERIODO_TOTAL) * integral_result
    return bn

# VERIFICAR LIMITES DE INTEGRACION
#-----------------------------SFT--------------------------
def serie_fourier_trigonometrica(n,w,t):
  acum = 0
  matrix1 = []
  for trozo in range(len(matrix)):
    acum += fourier_a0(matrix[trozo][0], float(matrix[trozo][1]), float(matrix[trozo][2]))
    for i in range(1, n + 1):
      an = fourier_an(matrix[trozo][0], float(matrix[trozo][1]),float(matrix[trozo][2]), i)
      bn = fourier_bn(matrix[trozo][0], float(matrix[trozo][1]),float(matrix[trozo][2]), i)
      acum += (an * np.cos(i*w * t)) + (bn * np.sin(i * w * t))

  return acum


#--------------------FUNCION ICE---------------------------

NRetorno = 0
def ICE():
  N = 1
  ICE_N = 0
  energia_final = calcular_energia()
  acumulado = 0
  llaves = 0
  a0 = 0
  an = 0
  bn = 0
  p1=0
  omega = 0
  a0Desp=0
  anDesp = 0
  bnDesp = 0
  inicio = 1
  matrisABNs=[]
  #------------------------------------------------------------------------
  while(True):
  #for i in range(1, 13):#ciclo para hacer N iteraciones manuales
  # evaluamos cada trozo de la funcion, e iteramos para sacar los n terminos de la sumatoria de An y Bn
  # al mismo tiempo almacenamos los valores de los ANs y BNs, para cara iteracion
    for trozo in range(len(matrix)):
      a0 += fourier_a0(matrix[trozo][0], float(matrix[trozo][1]), float(matrix[trozo][2]))
      for i in range(1, N+1):
        an = fourier_an(matrix[trozo][0], float(matrix[trozo][1]), float(matrix[trozo][2]), i)
        bn = fourier_bn(matrix[trozo][0], float(matrix[trozo][1]), float(matrix[trozo][2]), i)
        if(inicio == 1):
          arrayABNs = [float(an), float(bn)]
          matrisABNs.append(arrayABNs)
          if(len(matrisABNs)==len(range(1, N+1))):
            inicio += 1
        else:
          aaa=matrisABNs[i-1][0]+float(an)
          bbb=matrisABNs[i-1][1]+float(bn)
          matrisABNs[i-1][0] = aaa
          matrisABNs[i-1][1] = bbb
        acumulado += (an**2 + bn**2)
      anDesp = matrisABNs[-1][0]
      bnDesp = matrisABNs[-1][1]
      p1 = (a0**2)*(np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2])))#PERIODO TOTAL
      a0Desp = a0
    #TERMINA EL FOR DE CADA TROZO
    a0 = 0
    an = 0
    bn = 0

    llaves = ((p1) + (((np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2]))))/2)*acumulado)
    ICE_N = energia_final - llaves
    N +=1
    if(ICE_N <= 0.02*energia_final):
      NRetorno= N-1
      break
    else:
      matrisABNs = []
      inicio = 1
    acumulado = 0
    p1 = 0

  print("")
  print("")
  print("")
  print("RESULTADOS")
  print("PERIODO DE LA FUNCION: "+ str((np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2])))))
  print("")
  function = input("Desea enlisatar los An y Bn de las N iteraciones: Ingrese, SI para enlistar y NO/ENTER para proseguir: ")
  if (function == "NO" or function == "") and (not not matrisABNs):
    print("Coeficientes de Fourier para la N termino,  An: "+ str(anDesp)+" Bn: "+str(bnDesp))
  else:
    for fila in matrisABNs:
      print(fila)
  print("")
  print("Valor N para la serie: "+ str(NRetorno))
  print("")
  print("")
  omega = (2*np.pi)/(np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2])))
  t = np.linspace(0,(np.abs(float(matrix[0][1])) + np.abs(float(matrix[-1][2])))*2, 2000)  # Valores de tiempo de 0 a 2T
  plt.plot(t, serie_fourier_trigonometrica(NRetorno,omega,t))
  plt.xlabel('x')
  plt.ylabel('f(x) (Serie de Fourier)')
  plt.title('Serie de Fourier Trigonométrica')
  plt.grid(True)
  plt.show()

truncadoGen = 0
while(True):
  print("")
  print("")
  print("------------------ SUMATORIA TRUNCADA DE FOURIER ------------------ ")
  print("---------      Pablo Daniel Agreda Barrios - 17001969      ---------")
  print("---------      Jimmy Israel Mendoza Coquix - 17002925      ---------")
  print("--------------------------------------------------------------------")
  print("------------------      INSTRUCCIONES DE USO      ------------------")
  print("-      1) Por favor ingrese la funcion a trozos para calcular      -")
  print("-         la respectiva suma de fourier + ENTER, por favor         -")
  print("-         su funcion debe ser ingresada en terminos de x o t       -")
  print("-         todo esto en minusculas, por favor tomar en cuenta       -")
  print("-         que se aceptan unicamente funciones de una variable      -")
  print("-      2) Por favor ingrese el rango de operacion del trozo        -")
  print("-         ingresado, Rango Inicial + ENTER, Rango Final  +         -")
  print("-         ENTER, esto la cantidad de veces que este definida       -")
  print("-         la funcion, es decir repetir estos paso para la          -")
  print("-         cantidad de trozos de su funcion, tomar en cuenta        -")
  print("-         que para los rangos de operacion solo se aceptan         -")
  print("-         valores numericos.                                       -")
  print("-      3) Al finalizar el ingreso de los trozos de su funcion      -")
  print("-         omita el paso 1 y oprima ENTER, para salir de la         -")
  print("-         aplicacion, basta con ingresar SALIR en el paso 1        -")
  print("")
  print("")
  if func_input() == (-1):
    break
  else:
    ICE()
    reinicio()







------------------ SUMATORIA TRUNCADA DE FOURIER ------------------ 
---------      Pablo Daniel Agreda Barrios - 17001969      ---------
---------      Jimmy Israel Mendoza Coquix - 17002925      ---------
--------------------------------------------------------------------
------------------      INSTRUCCIONES DE USO      ------------------
-      1) Por favor ingrese la funcion a trozos para calcular      -
-         la respectiva suma de fourier + ENTER, por favor         -
-         su funcion debe ser ingresada en terminos de x o t       -
-         todo esto en minusculas, por favor tomar en cuenta       -
-         que se aceptan unicamente funciones de una variable      -
-      2) Por favor ingrese el rango de operacion del trozo        -
-         ingresado, Rango Inicial + ENTER, Rango Final  +         -
-         ENTER, esto la cantidad de veces que este definida       -
-         la funcion, es decir repetir estos paso para la          -
-         cantidad de trozos de 

KeyboardInterrupt: ignored